In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

#from pandas_profiling import ProfileReport
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
#from fancyimpute import KNN

In [2]:
churn = pd.read_csv('../data/churn.csv')
#Simple mean impute for ratings columns
imp = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imp.fit(churn[['avg_rating_by_driver','avg_rating_of_driver']])
churn_imp = imp.transform(churn[['avg_rating_by_driver','avg_rating_of_driver']])
pd.DataFrame(churn_imp)
churn[['avg_rating_by_driver','avg_rating_of_driver']]=churn_imp
churn#encode non-null values: https://towardsdatascience.com/preprocessing-encode-and-knn-impute-all-categorical-features-fast-b05f50b4dfaa 
encoder = OrdinalEncoder()
#imputer = KNN()
cat_cols = ['phone']


def encode(data):
    '''function to encode non-null data and replace it in the original data'''
    #retains only non-null values
    nonulls = np.array(data.dropna())
    #reshapes the data for encoding
    impute_reshape = nonulls.reshape(-1,1)
    #encode date
    impute_ordinal = encoder.fit_transform(impute_reshape)
    #Assign back encoded values to non-null values
    data.loc[data.notnull()] = np.squeeze(impute_ordinal)
    return data 

for columns in cat_cols:
    
    encode(churn[columns])
    
def churn_(df, column):
    dates = df[column]
    cutoff_date = '2014-06-01'
    array = []
    for date in dates:
        if date > cutoff_date:
            array.append(1)
        else:
            array.append(0)
    df['active'] = array
    return df

churn = churn_(churn, 'last_trip_date')
    
#KNN impute for phone column
imputer = KNNImputer(n_neighbors=5, weights="uniform")
churn_imp2=imputer.fit_transform(churn[['avg_rating_by_driver','avg_rating_of_driver','phone']])
churn[['avg_rating_by_driver','avg_rating_of_driver','phone']]=churn_imp2# creating instance of one-hot-encoder 
enc = OneHotEncoder(handle_unknown='ignore')
enc_df = pd.DataFrame(enc.fit_transform(churn[['city']]).toarray())
churn = churn.join(enc_df)
churn = churn.drop(['city'],axis =1)

#create feature with difference between signup and last trip
churn['last_trip_date_d'] = pd.to_datetime(churn['last_trip_date'])
churn['signup_date_d'] = pd.to_datetime(churn['signup_date'])
churn['date_delta'] = churn['last_trip_date_d']-churn['signup_date_d']
churn = churn.rename(columns = {0:'city_0', 1:'city_1',2:'city_2'},errors = 'raise')

def date_to_month(df, column_name):
    df[column_name] = pd.to_datetime(df[column_name])
    df['month '+ column_name] = df[column_name].dt.month
    return df

date_to_month(churn, 'signup_date')

churn = churn.drop(['signup_date'],axis =1)
date_to_month(churn, 'last_trip_date')
churn = churn.drop(['last_trip_date'], axis = 1)
churn = churn.drop(['last_trip_date_d'], axis = 1)
churn = churn.drop(['signup_date_d'], axis = 1)
# churn = churn.drop(['month_signup_date'], axis = 1)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [3]:
churn.columns

Index(['avg_dist', 'avg_rating_by_driver', 'avg_rating_of_driver', 'avg_surge',
       'phone', 'surge_pct', 'trips_in_first_30_days', 'luxury_car_user',
       'weekday_pct', 'active', 'city_0', 'city_1', 'city_2', 'date_delta',
       'month signup_date', 'month last_trip_date'],
      dtype='object')

In [4]:
churn['date_delta'] = churn['date_delta'].dt.days

In [5]:
churn.drop(columns=['date_delta', 'month last_trip_date'], inplace=True)

In [6]:
X = churn
y = churn.pop('active')


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.2, random_state=1)


In [8]:
gdbr = GradientBoostingClassifier(learning_rate=1,
                                  loss='deviance',
                                  n_estimators=100,
                                  random_state=1)





In [9]:
gdbr.fit(X_train, y_train)

GradientBoostingClassifier(learning_rate=1, random_state=1)

In [10]:
gdbr.score(X_test, y_test)

0.7878

In [ ]:
np.mean(gdbr.train_score_)

In [ ]:
for i in X:
    print('{predictor} has an importance of {importance}'.{predictor=i, 
                                                          importance=gdbr.feature_importances_})